In [1]:
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
import datetime as dt

In [2]:
# БЛОК С ИМПОРТОМ ДАННЫХ
data=pd.read_csv("./Data/casc-resto-small.csv",sep=',') # Считываем данные из файла
data['RKDate']=pd.to_datetime(data['RKDate'])
fix_date=(datetime.strptime('2017-07-01', '%Y-%m-%d'))
print(data)

     CustomerID  Restaurant     RKDate                    RegionName  \
0       2898197         391 2017-07-16  Москва и Московская область    
1       2903215          43 2015-04-07  Москва и Московская область    
2       2748887          43 2015-05-22  Москва и Московская область    
3       2862077          46 2015-03-05  Москва и Московская область    
4       2862077          46 2015-03-11  Москва и Московская область    
..          ...         ...        ...                           ...   
995     2899332         405 2015-08-30  Москва и Московская область    
996     2832254         405 2016-04-02  Москва и Московская область    
997     2751920         405 2016-04-09  Москва и Московская область    
998     2745527         405 2016-08-02  Москва и Московская область    
999     2887576         405 2016-08-19  Москва и Московская область    

               BrandsNames DishCategoryName  Quantity  SummBasic  \
0              TGI FRIDAYS      NON ALCOHOL         2          2   

In [12]:
# БЛОК С РАСЧЕТОМ ПОКАЗАТЕЛЕЙ
def agg(group): # функция, в которой считаются все показатели
    bought=0 # переменная для покупал/не покупал
    recency=10000
    spent = 0
    visitedTimes = 1
    nonCashSpent = 0
    visitedTimes1 = 1
    visitsBeforeFix=0
    firstVisitDate=fix_date
    MonetaryValue=0
    Mean_quantity=0
    QuantityForGroup=sum( group['Quantity'] )
    for index, row in group.iterrows(): # пробегаемся по датам в каждой группе 
        if row['RKDate'] <=  datetime.strptime('2017-12-31', '%Y-%m-%d')  and row['RKDate'] >= fix_date: # сравниваем даты
            bought=1 # покупка была совершена в промежуток времени 
            
        if row['RKDate'] <= fix_date and (fix_date - row['RKDate']).days < recency: 
            recency = (fix_date - row['RKDate']).days # считаем количество дней от даты покупки/визита до фиксированной даты
                
        if row['RKDate'] <= fix_date:
            visitsBeforeFix=visitsBeforeFix+1 # считаем дату самого недавнего визита до фиксированной даты
            spent+=row['SummAfterPointsUsage'] # считаем сколько всего было потрачено
        if row['RKDate']<= firstVisitDate:
            firstVisitDate=row['RKDate'] # считаем дату первого визита 
            
        #if (row['RKDate'] <= fix_date):           
            

    if visitsBeforeFix == 0: # если не было визитов до фиксированной даты, то частота равна 0
        frequency=0
    months  = 1 + (relativedelta.relativedelta(fix_date,firstVisitDate)).months # считаем сколько месяцев прошло от первого визита до фиксированной даты
    if months!=0:
        frequency= visitsBeforeFix/ months # считаем частоту
    if visitsBeforeFix != 0:
        MonetaryValue= spent / visitsBeforeFix # считаем средний чек покупки за время до фиксированной даты  
        Mean_quantity=QuantityForGroup/visitsBeforeFix
     # среднее количество купленного товара до фиксированной даты, собственный показатель
    
    return bought,recency,frequency,MonetaryValue,Mean_quantity 

    

def func(group): #функция, в которой все посчитанные показатели собираются в один массив
    bought, recency,frequency,MonetaryValue, Mean_quantity=agg(group)
    d = {}
    d['Bought_Or_Not'] = bought
    d['Recency'] = recency
    d['Frequency'] = frequency
    d['Monetary_Value'] = MonetaryValue
    d['Mean_Quantity'] = Mean_quantity
   
    return pd.Series(d, index=['Bought_Or_Not', 'Recency','Frequency','Monetary_Value','Mean_Quantity'])

groupedData = data.groupby("CustomerID").apply(func) # группируем записи по клиенту и применяем функцию, в которой все считается
aggregatedData= groupedData.reset_index()
print(aggregatedData)



     CustomerID  Bought_Or_Not  Recency  Frequency  Monetary_Value  \
0       2728549            0.0    793.0   0.500000        7.500000   
1       2729112            0.0    818.0   0.333333       10.000000   
2       2729528            0.0    856.0   0.200000       10.000000   
3       2729539            0.0    837.0   0.250000        5.000000   
4       2729810            0.0    863.0   0.333333        6.250000   
..          ...            ...      ...        ...             ...   
458     2912433            0.0    387.0   1.000000        6.666667   
459     2912538            0.0    710.0   0.083333        6.890000   
460     2912639            0.0    823.0   0.250000        5.000000   
461     2912991            1.0  10000.0   0.000000        0.000000   
462     2913068            1.0  10000.0   0.000000        0.000000   

     Mean_Quantity  
0         1.333333  
1         1.000000  
2         1.000000  
3         1.000000  
4         1.500000  
..             ...  
458       1.

In [13]:
# БЛОК СО СЛИЯНИЕМ ДАННЫХ
dataToMergeWith =pd.read_csv("./Data/CASC_constant.csv",sep=',') # Считываем данные из файла для слияния
dataToMergeWith['ActivationDate'] = pd.to_datetime(dataToMergeWith['ActivationDate'])
MergedData=pd.merge(aggregatedData, dataToMergeWith, left_on='CustomerID', right_on='CustomerId',how='inner') # Делаем слияние двух таблиц
del MergedData['CustomerId']
del MergedData['CustomerID']
Sex=pd.get_dummies(MergedData['Sex'], drop_first=True) # создадим новый столбец, в котором используем фиктивные переменные: мужской пол =1 ,женский пол = 0

MergedData.drop(['Sex'],axis=1,inplace=True) # Удаляем столбец с полом, потому что там прописано м и ж, а нам нужно 1 или 0

MergedData=pd.concat([MergedData,Sex],axis=1) # присоединяем новый столбец с полом в нужном формате к нашей таблице

MergedData['ActivationDate'] = pd.to_datetime(MergedData['ActivationDate']) # переведем формат из строки в дату

MergedData['ActivationDate']=MergedData['ActivationDate'].map(dt.datetime.toordinal) # сделаем дату числом
MergedData['Age'] = MergedData['Age'].fillna(MergedData['Age'].mean())
print(MergedData)

     Bought_Or_Not  Recency  Frequency  Monetary_Value  Mean_Quantity  \
0              0.0    793.0   0.500000        7.500000       1.333333   
1              0.0    818.0   0.333333       10.000000       1.000000   
2              0.0    856.0   0.200000       10.000000       1.000000   
3              0.0    837.0   0.250000        5.000000       1.000000   
4              0.0    863.0   0.333333        6.250000       1.500000   
..             ...      ...        ...             ...            ...   
458            0.0    387.0   1.000000        6.666667       1.000000   
459            0.0    710.0   0.083333        6.890000       1.000000   
460            0.0    823.0   0.250000        5.000000       1.000000   
461            1.0  10000.0   0.000000        0.000000       0.000000   
462            1.0  10000.0   0.000000        0.000000       0.000000   

     ActivationDate   Age  SubscribedEmail  SubscribedPush  Male  
0            735600  32.0            False           Fal

In [14]:
# БЛОК С ОБУЧЕНИЕМ МОДЕЛИ
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

X_data=MergedData.drop('Bought_Or_Not',axis=1) # исключаем из данных зависимую переменную

x_train, x_test, y_train, y_test = train_test_split(X_data, MergedData['Bought_Or_Not'], test_size=0.2, random_state=42) # разделим данные, тестовая выборка составит 20%
logreg=LogisticRegression() # создадим логистическую регрессию
logreg.fit(x_train, y_train) # применим функцию fit на обучающей выборке
y_pred = logreg.predict(x_test) # применим функцию predict для предсказания вероятности прихода покупателя
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred)) 

[[89  0]
 [ 1  3]]
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        89
         1.0       1.00      0.75      0.86         4

    accuracy                           0.99        93
   macro avg       0.99      0.88      0.93        93
weighted avg       0.99      0.99      0.99        93



C:\Users\tagil\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [27]:
print(data)

     CustomerID  Restaurant     RKDate                    RegionName  \
0       2898197         391 2017-07-16  Москва и Московская область    
1       2903215          43 2015-04-07  Москва и Московская область    
2       2748887          43 2015-05-22  Москва и Московская область    
3       2862077          46 2015-03-05  Москва и Московская область    
4       2862077          46 2015-03-11  Москва и Московская область    
..          ...         ...        ...                           ...   
995     2899332         405 2015-08-30  Москва и Московская область    
996     2832254         405 2016-04-02  Москва и Московская область    
997     2751920         405 2016-04-09  Москва и Московская область    
998     2745527         405 2016-08-02  Москва и Московская область    
999     2887576         405 2016-08-19  Москва и Московская область    

               BrandsNames DishCategoryName  Quantity  SummBasic  \
0              TGI FRIDAYS      NON ALCOHOL         2          2   